# Preprocessing for Nitrogen Loading Model (Nload)

## Objectives

Within subwatersheds of the Great, Green, and Bourne ponds drainages, reclassify Massachusetts 2016 land cover layer to include cranberry agriculture into 13 categories for use in Nload. Repeat this for baseline conditions, and several scenarios.

Future scenarios could include increasing population and impervious area by displacing natural areas. Growth of cranberry agriculture in uplands with modern cranbery hybrids. Retirement and/or restoration of older and flowthrough bogs. Replacement of older flowthrough bogs with new hybdrids. Upgrades to wastewater treatment facilities. 


## References 
Kennedy, C. D., & Hoekstra, B. R. (2021). Measuring and modeling nitrogen export from cranberry farms. Ecosphere, 12(12), 1–15. https://doi.org/10.1002/ecs2.3686

Valiela, I., Geist, M., McClelland, J., & Tomasky, G. (2000). Nitrogen loading from watersheds to estuaries: Verification of the Waquoit Bay Nitrogen Loading Model. Biogeochemistry, 49(3), 277–293. https://doi.org/10.1023/A:1006345024374

Valiela, I., Collins, G., Kremer, J., Lajtha, K., Geist, M., Seely, B., … Sham, C. H. (1997). Nitrogen loading from coastal watersheds to receiving estuaries: New method and application. Ecological Applications, 7(2), 358–380. https://doi.org/10.1890/1051-0761(1997)007[0358:NLFCWT]2.0.CO;2

## Set up environment

In [319]:
# Load dependancies ***************************************
import arcpy as ap
import os, glob, re, sys
import pandas as pd
import numpy as np
#import scipy.stats

# Set up arcpy environment ******************************
# input data directory
idr = "C:\\Workspace\\Geodata\\Massachusetts\\"
# working directory this should be where arcgis project is located
wdr = "C:\\Workspace\\Geodata\\Nload\\"
# output directory
odr = os.path.join(wdr,'outputs')
# arcpy settings
ap.env.workspace = wdr # set arcpy environment working directory
aprx = arcpy.mp.ArcGISProject("Current") # current project
ap.CheckOutExtension("Spatial") # check out spatial extension
ap.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 UTM Zone 19N") # define cordinate system
# Allow overwriting of output
arcpy.env.overwriteOutput = True
#ap.SetProgressor("step", "processing...", step_value = 5)

Tip: see this [page](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/setprogressor.htm) for tips on arcpy settings. 

Tip: Use `os.path.join` to safely concatenate file paths. To do this enter folder names in quotes separated by commas.

Tip: Test in the geoprocessing tab first, then, once working, can copy python code from the geoprocessing tab when you click on the arrow next to `run`!

In [320]:
# check that directories exist
if os.path.exists(wdr) is False:
    sys.exit(wdr+" does not exist")
if os.path.exists(idr) is False:
    sys.exit(idr+" does not exist")
if os.path.exists(odr):
    print(odr+" already exists")
else:
    os.mkdir(odr)

C:\Workspace\Geodata\Nload\outputs already exists


Tip: Use `arcpy.FeatureSet` to add an input feature as an object to the current environment. Alternatively, `arcpy.management.MakeFeatureLayer` can be used to add input feature as a layer to the current project. The added layer should appear on the map. `arcpy.Copyfeatures_management` works in a similar way to `MakeFeatureLayer`. 

## Load input files

In [321]:
# Add Input Data *********************************** 
# Massachusetts 2016 Land Use Land Cover Layer 
path = os.path.join(idr,"lclu_gdb","MA_LCLU2016.gdb","LANDCOVER_LANDUSE_POLY")
MA_LCLU2016 = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_lclu16")

# Massachusetts cranberry agriculture (source: Jim McLaughlin)
# this needs to be updated
path = os.path.join(idr,'WMAbogsDRAFT2013\\WMAbogsDRAFT2013.shp')
os.path.exists(path)
arcpy.CopyFeatures_management(path,"in_bogs")
bogs = ap.FeatureSet(path)
#ap.management.MakeFeatureLayer(path, "lyr_ggbp")
path = os.path.join(idr,'Wareham_Bogs_v3\\WarehamBogs_master_v3.shp')
os.path.exists(path)
arcpy.CopyFeatures_management(path,"in_wareham_bogs")
wareham_bogs = ap.FeatureSet(path)

#ap.management.MakeFeatureLayer(path, "lyr_ggbp")
path = os.path.join('in_ggbp_Clip')
os.path.exists(path)
#arcpy.CopyFeatures_management(path,"in_ggbp_bogs")
ap.management.MakeFeatureLayer(path,"in_ggbp_bogs")
ggbp_bogs = ap.FeatureSet(path)

# Subcatchments of Agawam (Wareham) with terminus classifications
path = os.path.join(idr,"agawam_WS","agawam_subw_0309.shp")
agawam = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_agawam")

# Subcatchments of Great, Green, and Bourne Ponds with terminus classifications
path = os.path.join(idr,"GGBsheds","Great_Green_Bourne_Ponds.shp")
os.path.exists(path)
ggbp = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_ggbp")



<Result 'in_ggbp'>

## Main Program

### Part 1: Combine spatial layers

`arcpy.analysis.Intersect` (A) land use and (B) subwatershed Layer. 

In [322]:
# Intersect MA land use layer with subwatershed layer
inFeatures = [MA_LCLU2016, ggbp]

In [323]:
intersectOutput = "out_ggbp_lclu16"

In [324]:
arcpy.analysis.Intersect(inFeatures, intersectOutput, "", "", "INPUT")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_ggbp_lclu16.shp'>

Spatial (A) join cranberry and (B) intersection of subwatershed land use layers. 

In [325]:
inFeat = intersectOutput
identFeat = "in_ggbp_bogs"
identOutput = "out_lclu16_ggbp_cran_ident"

In [326]:
arcpy.analysis.Identity(inFeat, identFeat, identOutput, "ALL", None, "NO_RELATIONSHIPS")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_lclu16_ggbp_cran_ident.shp'>

Calculate area of polygons in hectares and export attribute tables to pandas dataframe

In [327]:
def fn_Calculate_Area_Ha(fc):
    # calculate area in hectares and add column to attribute table
    arcpy.management.CalculateGeometryAttributes(fc,
                                             "Area_Ha AREA_GEODESIC",
                                             '', "HECTARES", None, "SAME_AS_INPUT")

# path to feature class
fc1 = "in_ggbp"
fn_Calculate_Area_Ha(fc1)

In [328]:
fc2 = identOutput
fn_Calculate_Area_Ha(fc2)

In [329]:
def fn_featureClass_Attributes_to_DataFrame(fc):
    # list of field names that excludes the shape geometry (including geometry messes this up)
    flds = [fld.name for fld in arcpy.ListFields(fc) if fld.name != arcpy.Describe(fc).shapeFieldName]
    arr = arcpy.da.FeatureClassToNumPyArray(fc,flds)
    df = pd.DataFrame(arr)
    return(df)

in_df1 = fn_featureClass_Attributes_to_DataFrame(fc1).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

In [330]:
in_df2 = fn_featureClass_Attributes_to_DataFrame(fc2).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

In [331]:
# check that area of the inputs and outputs match closely
Total_Area1 = in_df1.Area_Ha.sum()
Total_Area2 = in_df2.Area_Ha.sum()
print(fc1+" Area (Ha):"+str(Total_Area1))
print(fc2+" Area (Ha):"+str(Total_Area2))
print("Difference in Area:"+str(Total_Area1 - Total_Area2)) # difference should be close to 0.0

in_ggbp Area (Ha):4582.647164184
out_lclu16_ggbp_cran_ident Area (Ha):4582.646524428548
Difference in Area:0.0006397554516297532


In [332]:
# write attribute tables to csv
in_df1.to_csv(os.path.join(odr,fc1+".csv"))
in_df2.to_csv(os.path.join(odr,fc2+".csv"))

### Part 1: Reclassify combined layers for Nload Scenarios

Read excel tables with subwatershed land use area and terminus information

In [333]:
# Subwatershed land use area
inname1 = "in_ggbp"
inname2 = "out_lclu16_ggbp_cran_ident"
in_df1 = pd.read_csv(os.path.join(odr,inname1+'.csv')).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

# Subwatershed catchment ids and terminus information
in_df2 = pd.read_csv(os.path.join(odr,inname2+'.csv')).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus','Bog_Type':'Bog_type'})

sys:1: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


In [334]:
display(in_df1)
display(in_df2)

,Unnamed: 0,FID,SUB,SHED_NAME,Shape_area,Shape_len,Terminus,Area_Ha
0,0,0,4,Deep Pond GT10,7.178481e+05,6853.937704,p,71.783290
1,1,1,6,Crooked Pond GT10,5.307302e+05,6260.229310,p,53.071712
2,2,2,2,Coonamessett Pond LT10,3.528562e+06,8890.501888,p,352.846203
3,3,3,5,Deep Pond LT10,6.521168e+05,4807.831317,p,65.209861
4,4,4,3,Round Pond,3.702084e+05,4271.582585,p,37.019734
5,5,5,7,Crooked Pond LT10,7.813053e+05,6005.219838,p,78.128171
6,6,6,9,Round Pond,8.723918e+05,5747.750306,p,87.236109
7,7,7,15,Flax Pond GT10,2.288562e+05,3451.244741,p,22.884859
8,8,8,12,Mares Pond GT10,5.452401e+05,6000.947958,p,54.522097
9,9,9,10,Jenkins Pond,2.133851e+06,9845.974167,p,213.377284


,Unnamed: 0,FID,FID_out_gg,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_Great_,SUB,SHED_NAME,Shape_ar_1,Shape_len,Terminus,Area_Ha,FID_in_ggb,ID,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Shape_Le_1,Shape_Ar_2,Bog_type
0,0,0,0,19202091,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,59.355339,178.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017850,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.00,,,1899-12-30,,,0.000,0.000,,,,0.000,,,,0.000000,0.000000,
1,1,1,1,19202092,Developed Open Space,5,Tax exempt,9,900,FEE,2011,261,R13C25,497.697006,1115.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.111562,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.00,,,1899-12-30,,,0.000,0.000,,,,0.000,,,,0.000000,0.000000,
2,2,2,2,19202150,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,182.752309,593.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.059350,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.00,,,1899-12-30,,,0.000,0.000,,,,0.000,,,,0.000000,0.000000,
3,3,3,3,19202201,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,226.793939,1051.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.105150,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.00,,,1899-12-30,,,0.000,0.000,,,,0.000,,,,0.000000,0.000000,
4,4,4,4,19202221,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,62.577164,178.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017862,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.00,,,1899-12-30,,,0.000,0.000,,,,0.000,,,,0.000000,0.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84220,84220,84220,82317,22780276,Cultivated,6,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,499.295570,7705.037920,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.762434,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,,42209609,238739.793,8156.279,,,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough
84221,84221,84221,82666,22781347,Palustrine Emergent Wetland,15,Open land,2,932,FEE,2017,96,R14C25,4.276612,0.530611,25,19,Lower Coonamessett River,3.521906e+06,11291.923980,s,0.000053,6,0,42209610,Town of Falmouth,Old Meeting House Rd.,Falmouth,4,Middle/lower bog,24.4,0.0,0.0,0.0,24.43,Jim McLaughlin,BRP_WMA,2009-05-13,,42209609,98550.168,4547.228,,,Town of Falmouth,24.352,Cape Cod,active,584,4546.272832,98508.748324,flowthrough
84222,84222,84222,82836,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,20,16,Flax Pond LT10,7.571123e+05,5773.200642,p,0.029896,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,,42209609,238739.793,8156.279,,,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough
84223,84223,84223,82837,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.018259,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,,42209609,238739.793,8156.279,,,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough


In [335]:
# Subwatershed housing units, driveways, septics etcs. 
in_df3 = pd.read_excel(r'C:\Users\Adrian.Wiegman\OneDrive - USDA\Research\Nload\Inputs\Nload_Watershed_Info.xlsx','ggbp_info',comment="#")
display(in_df3)

df1 = in_df1[['SUB','SHED_NAME','Terminus','Area_Ha']]
display(df1)
df1.SepticUnits = 0
df1 = df1.rename(columns={'SHED_NAME':'SubW_Name'})
#df1 = df1.merge(in_df3[['SUB','Terminus','SepticUnits']])
display(df1)

,SubW_Name,SUB,Area_Ha,Terminus,CC7_Ha,CC3_Ha,CC2_Ha,People_On_Septic,Imp_NR_Ac,NC_cropland_Ac,Unnamed: 10,Buildings,SepticUnits,Unnamed: 13,Unnamed: 14
0,Deep Pond GT10,4,125.113762,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Crooked Pond GT10,6,125.113762,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Coonamessett Pond LT10,2,118.460125,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deep Pond LT10,5,125.113762,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Round Pond,3,324.010057,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Crooked Pond LT10,7,125.113762,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Round Pond,3,324.010057,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Flax Pond GT10,15,59.632592,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Mares Pond GT10,12,581.102837,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Jenkins Pond,10,741.605861,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,SUB,SHED_NAME,Terminus,Area_Ha
0,4,Deep Pond GT10,p,71.783290
1,6,Crooked Pond GT10,p,53.071712
2,2,Coonamessett Pond LT10,p,352.846203
3,5,Deep Pond LT10,p,65.209861
4,3,Round Pond,p,37.019734
5,7,Crooked Pond LT10,p,78.128171
6,9,Round Pond,p,87.236109
7,15,Flax Pond GT10,p,22.884859
8,12,Mares Pond GT10,p,54.522097
9,10,Jenkins Pond,p,213.377284


,SUB,SubW_Name,Terminus,Area_Ha
0,4,Deep Pond GT10,p,71.783290
1,6,Crooked Pond GT10,p,53.071712
2,2,Coonamessett Pond LT10,p,352.846203
3,5,Deep Pond LT10,p,65.209861
4,3,Round Pond,p,37.019734
5,7,Crooked Pond LT10,p,78.128171
6,9,Round Pond,p,87.236109
7,15,Flax Pond GT10,p,22.884859
8,12,Mares Pond GT10,p,54.522097
9,10,Jenkins Pond,p,213.377284


Initialize a dataframe for Land cover (LUC) area by Subcatchment (SUB) with values for use class 2 (lawns), 3 (non cranberry ag), 7 (impervous, non-road). 

In [336]:
nLUCs = 13
SUBs =  list(in_df1['SUB'])*nLUCs
print(len(list(in_df1['SUB'])))
LUCs = [i+1 for i in range(nLUCs)]*len(list(in_df1['SUB']))
#print(LUCs)
df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
df_SUB_LUC
df_SUB_LUC
display(df_SUB_LUC)
df_merged = None
'''
coverclasses = [2,3,7]
for i in coverclasses:
    df_temp = in_df3
    df_temp['LUC'] = i
    df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
    df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'left')
display(df_merged)
'''

i = 2
j = 13
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_temp[['SUB','LUC','Area_Ha']]
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

i = 3 # area to be added
j = 4 # area to be subtracted from 
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

i = 7
j = 6
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

df_SUB_LUC_merged = pd.merge(df_SUB_LUC,df_merged,'left')
display(df_SUB_LUC_merged)
print(len(df_SUB_LUC_merged['SUB'].unique()))
df_SUB_LUC_merged.to_csv("df_SUB_LUC_merged.csv")
print(df_SUB_LUC_merged.sort_values(['LUC','SUB']))

33


,SUB,LUC
0,4,1
1,6,2
2,2,3
3,5,4
4,3,5
...,...,...
424,31,9
425,24,10
426,27,11
427,32,12


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
5,7,2,NaN
6,3,2,NaN
7,15,2,NaN
8,12,2,NaN
9,10,2,NaN


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
...,...,...,...
61,31,13,NaN
62,24,13,NaN
63,27,13,NaN
64,32,13,NaN


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
...,...,...,...
94,31,3,NaN
95,24,3,NaN
96,27,3,NaN
97,32,3,NaN


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
...,...,...,...
127,31,4,NaN
128,24,4,NaN
129,27,4,NaN
130,32,4,NaN


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
...,...,...,...
160,31,7,NaN
161,24,7,NaN
162,27,7,NaN
163,32,7,NaN


,SUB,LUC,Area_Ha
0,4,2,NaN
1,6,2,NaN
2,2,2,NaN
3,5,2,NaN
4,3,2,NaN
...,...,...,...
193,31,6,NaN
194,24,6,NaN
195,27,6,NaN
196,32,6,NaN


,SUB,LUC,Area_Ha
0,4,1,NaN
1,6,2,NaN
2,2,3,NaN
3,5,4,NaN
4,3,5,NaN
...,...,...,...
430,31,9,NaN
431,24,10,NaN
432,27,11,NaN
433,32,12,NaN


33
     SUB  LUC  Area_Ha
251    1    1      NaN
303    2    1      NaN
172    3    1      NaN
0      4    1      NaN
237    5    1      NaN
..   ...  ...      ...
395   29   13      NaN
434   30   13      NaN
263   31   13      NaN
64    32   13      NaN
329   33   13      NaN

[435 rows x 3 columns]


When septic unit data are unavailable. We estimate septic units from the following information sources.

Estimate fraction of total population (k_Pop) living in each subwatershed using one of two methods:
1. Impervious Area (USEGENCODE == 2): dividing the total impervious area by the impervious area within each subwatershed. 
2. Residential Use Codes (USEGENCODE == 10,11,12,13): classify polygons as housing units if they fall under a residential use code

It may be worthwile to add some uncertainty over how many housing units 

https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf

In [337]:
in_df2["ResidentialUnit"] = (in_df2.COVERCODE == 2) & np.isin(in_df2.USEGENCODE,[10,11,12,13])

USE_CODE_DICT = {
# Residences 
101: 1, # '......Single Family'
102: 5, #'......Condominium' between 
103: 20, #'......Mobile Home (includes land used for purpose of a mobile home park)' 
104: 2, #'......Two-Family'
105: 3, #'......Three-Family'
106: 0, #'......Accessory Land with Improvement - garage'
107: 0, #'......(Intentionally left blank)'
108: 0, #'......(Intentionally left blank)
109: 2, #'......Multiple Houses on one parcel (for example, a single and a two-family on one parcel)'
# Apartments
111: 6,#'......Four to Eight Units'
112: 10#'......More than Eight Unit'
}

totalHousingUnits = in_df2['ResidentialUnit'].sum()
print(totalHousingUnits)
print(totalHousingUnits*2.43)

df_HU_SUB = in_df2.groupby('SUB')['ResidentialUnit'].sum().reset_index(name='ResUnits')
df_HU_SUB['ResImp_Fraction'] = df_HU_SUB['ResUnits']/totalHousingUnits
display(df_HU_SUB)

15314
37213.020000000004


,SUB,ResUnits,ResImp_Fraction
0,1,63,0.004114
1,2,311,0.020308
2,3,20,0.001306
3,4,0,0.000000
4,5,110,0.007183
5,6,21,0.001371
6,7,344,0.022463
7,8,39,0.002547
8,9,116,0.007575
9,10,604,0.039441


In [338]:
totalImpervious = in_df2.loc[in_df2['COVERCODE']==2, 'Area_Ha'].sum()
df_imp_sub = in_df2.loc[in_df2['COVERCODE']==2].groupby('SUB')['Area_Ha'].sum().reset_index(name='Area_Ha')
df_imp_sub['Impervious_Fraction'] = df_imp_sub['Area_Ha']/totalImpervious
print(df_imp_sub)

    SUB    Area_Ha  Impervious_Fraction
0     1  45.003788             0.092609
1     2  16.395758             0.033739
2     3   1.299574             0.002674
3     4   6.086647             0.012525
4     5   3.405454             0.007008
5     6   1.623396             0.003341
6     7   7.785850             0.016022
7     8   1.264153             0.002601
8     9   4.963642             0.010214
9    10  13.854318             0.028509
10   11   1.963666             0.004041
11   12   2.229241             0.004587
12   13   1.488782             0.003064
13   14   2.244471             0.004619
14   15   2.181547             0.004489
15   16   6.859169             0.014115
16   17  12.990437             0.026732
17   18  28.902847             0.059476
18   19  59.217469             0.121858
19   20   4.582683             0.009430
20   21  12.997854             0.026747
21   22  13.540230             0.027863
22   23   6.081851             0.012515
23   24  73.361005             0.150962


In [339]:
df1['k_Pop'] = None
df1 = pd.merge(df1,df_imp_sub[['SUB','Impervious_Fraction']])
df1 = pd.merge(df1,df_HU_SUB[['SUB','ResUnits','ResImp_Fraction']])

# how well do these metrics align
#import matplotlib.pyplot as plt
#df1.plot('Housing_Fraction','Impervious_Fraction','scatter')
#plt.show()

df1['k_Pop'] = df1.loc[:,'ResImp_Fraction']
display(df1)

,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,4,Deep Pond GT10,p,71.783290,0.000000,0.012525,0,0.000000
1,6,Crooked Pond GT10,p,53.071712,0.001371,0.003341,21,0.001371
2,2,Coonamessett Pond LT10,p,352.846203,0.020308,0.033739,311,0.020308
3,5,Deep Pond LT10,p,65.209861,0.007183,0.007008,110,0.007183
4,3,Round Pond,p,37.019734,0.001306,0.002674,20,0.001306
5,7,Crooked Pond LT10,p,78.128171,0.022463,0.016022,344,0.022463
6,9,Round Pond,p,87.236109,0.007575,0.010214,116,0.007575
7,15,Flax Pond GT10,p,22.884859,0.001437,0.004489,22,0.001437
8,12,Mares Pond GT10,p,54.522097,0.002808,0.004587,43,0.002808
9,10,Jenkins Pond,p,213.377284,0.039441,0.028509,604,0.039441


Reclassify land use using Nload cover codes for default scenario

In [340]:
# GENERATE DICTIONARY OF CONDITIONS FOR RECLASSIFYING LAND USE

# specific active cranberry: where other farming makes up a non zero portion of cultivated and hay/pasture cover types
specific_active_cranberry = '(np.isin(df.CropStatus,"active"))'

# general active cranberry: use where active cranberry makes up over 99% of COVERCODE 6 and 7. 
general_active_cranberry = '((np.isin(df.CropStatus,"active"))|((df.COVERCODE == 21) & (df.USEGENCODE==7))|(np.isin(df.COVERCODE,[6,7])))'

active_cranberry = specific_active_cranberry
abandoned_cranberry = 'np.isin(df["FID"],df[df.CropStatus == "abandoned"]["FID"])'
flowthrough = 'np.isin(df["FID"],df[df.Bog_type == "flowthrough"]["FID"])'
#abandoned_cranberry = '(np.isin(df.CropStatus,"abandoned"))'
#flowthrough = '(np.isin(df.Bog_type,"flowthrough"))'
waterbody = '(np.isin(df.COVERCODE,[22,21]) & np.invert('+active_cranberry+'))'
# condition dictionary
SPECIFIC = {
    # THE FIRST APPEARING CONDITION TAKES PRIORITY IN NP.SELECT
    # key (LUC): value (condition string)
     # 4: "Agriculture, Active Cranberry, Flowthrough"
    4: active_cranberry+' & '+flowthrough,
    # 5: "Agriculture, Active Cranberry, Non Flowthrough"
    5: active_cranberry+' & (np.invert('+flowthrough+'))',
    # 11: "Retired Cranberry, Flowthrough"
    11: abandoned_cranberry+' & '+flowthrough,
    # 12: "Retired Cranberry, Non flowthrough"
    12: abandoned_cranberry+' & np.invert('+flowthrough+')',
    # 1: "Natural Uplands"
    1: '(df.COVERCODE >= 8) & (df.COVERCODE <= 12)',
     # 6: "Impervious, Roads"
    6: '(df.COVERCODE == 2)',
    # 3: "Agriculture, Non Cranberry"
    3: '(np.isin(df.COVERCODE,[6,7])) & (np.invert('+active_cranberry+'))',
    # 8: "Recieving Body (Estuary)"
    8: '(df.SUB == "0") & ((df.COVERCODE == 23)|'+waterbody+')', # note that SUB is a string
    # 9: "Freshwater Ponds and Lakes"
    9: '(df.SUB != "0") &' + waterbody,
    # 10: "Wetlands"
    10: '(df.COVERCODE >= 13) & (df.COVERCODE <= 18)',
    # 2: "Mowed Areas (Lawns, Sports Fields, and Golf Courses)"
    2: 'df.COVERCODE is [False]', # this will be calculated separately
    # 7: "Impervious, Non-Roads"
    7: 'df.COVERCODE is [False]', # this will be calculated separately
    # 13: "Other (Bare land or Shoreline)"
    13: 'df.COVERCODE is [False]'}

GENERAL = {
    # THE FIRST APPEARING CONDITION TAKES PRIORITY IN NP.SELECT
    # key (LUC): value (condition string)
     # 4: "Agriculture, Active Cranberry, Flowthrough"
    4: active_cranberry+' & '+flowthrough,
    # 5: "Agriculture, Active Cranberry, Non Flowthrough"
    5: active_cranberry+' & (np.invert('+flowthrough+'))',
    # 11: "Retired Cranberry, Flowthrough"
    11: abandoned_cranberry+' & '+flowthrough,
    # 12: "Retired Cranberry, Non flowthrough"
    12: abandoned_cranberry+' & np.invert('+flowthrough+')',
    # 1: "Natural Uplands"
    1: 'df.COVERCODE.between(8, 12, inclusive="both")',
     # 6: "Impervious, Roads"
    6: '(df.COVERCODE == 2) & (df.USEGENCODE == 55)',
    ## 3: "Agriculture, Non Cranberry"
    #3: 'df.COVERCODE == [False]',
    # 8: "Recieving Body (Estuary)"
    8: '(df.SUB == "0") & ((df.COVERCODE == 23)|'+waterbody+')', # note that SUB is a string
    # 9: "Freshwater Ponds and Lakes"
    9: '(df.SUB != "0") &' + waterbody,
    # 10: "Wetlands"
    10: 'df.COVERCODE.between(13, 18, inclusive="both")',
    # 2: "Mowed Areas (Lawns, Sports Fields, and Golf Courses)"
    2: 'df.COVERCODE == 5',
    # 7: "Impervious, Non-Roads"
    7: '(df.COVERCODE == 2) & (df.USEGENCODE != 55)',
    # 13: "Other (Bare land or Shoreline)"
    13: 'np.isin(df.COVERCODE,[19,20])'
}

BASELINE = GENERAL    

# checking condtions 
df = pd.DataFrame({"FID":[0,1,2,3],
                   "COVERCODE":[21,21]*2,
                   "SUB":["0","1"]*2,
                   "CropStatus":["active"]*2 + ["abandoned"]*2,
                   "Bog_type":["","flowthrough"]*2,
                   "USEGENCODE":[0,0,0,0],
                   "USE_CODE":[0,0,0,0]})
df = pd.DataFrame({"FID":[4,5,6,7],
                   "COVERCODE":[2,5]*2,
                   "SUB":["0","1"]*2,
                   "CropStatus":[None]*2 + [None]*2,
                   "Bog_type":[None,None]*2,
                   "USEGENCODE":[55,55,0,0],
                   "USE_CODE":[0,0,0,0]})
df = pd.DataFrame({"FID":[8,9,10,11],
                   "COVERCODE":[9,15,19,22],
                   "SUB":["0","1"]*2,
                   "CropStatus":[None]*2 + [None]*2,
                   "Bog_type":[None,None]*2,
                   "USEGENCODE":[55,55,0,0],
                   "USE_CODE":[0,0,0,0]})
df["8"]= eval(BASELINE[8])
df["9"]= eval(BASELINE[9])
df["11"]= eval(BASELINE[11])
df["12"]= eval(BASELINE[12])
df["2"]= eval(BASELINE[2])
df["6"]= eval(BASELINE[6])
df["7"]= eval(BASELINE[7])
df["13"]= eval(BASELINE[13])
df["10"]= eval(BASELINE[10])
df["1"]= eval(BASELINE[1])

condlist = "["+", ".join(list(BASELINE.values()))+"]"
choicelist = list(BASELINE.keys())
df["LUC"] = np.select(eval(condlist),choicelist,13)
print(df)

   FID  COVERCODE SUB CropStatus Bog_type  ...      7     13     10      1  LUC
0    8          9   0       None     None  ...  False  False  False   True    1
1    9         15   1       None     None  ...  False  False   True  False   10
2   10         19   0       None     None  ...  False   True  False  False   13
3   11         22   1       None     None  ...  False  False  False  False    9

[4 rows x 18 columns]


In [341]:
df = in_df2
#df['Bog_type'] = 'flowthrough'
for i in list(BASELINE.keys()): 
    print("True is in "+str(i)+"....",True in eval(BASELINE[i]))

True is in 4.... True
True is in 5.... True
True is in 11.... False
True is in 12.... False
True is in 1.... True
True is in 6.... True
True is in 8.... True
True is in 9.... True
True is in 10.... True
True is in 2.... True
True is in 7.... True
True is in 13.... True


In [342]:
# declare land use class (LUC) reclasssification function
def fn_reclassify_LUC(df,dictionary,default=13):
    condlist = "["+", ".join(list(dictionary.values()))+"]"
    choicelist = list(dictionary.keys())
    #print(condlist)
    #print(choicelist)
    df["LUC"] =  np.select(eval(condlist),choicelist,default)
    return(df)

In [343]:
# execute reclassification subroutine
df2 = fn_reclassify_LUC(in_df2,BASELINE)
print("Number of LUC == 13:",sum(df2['LUC']==2))
# replace whitespace data with NaN
df2 = df2.replace(r'^\s*$', np.nan, regex=True)
print(df2.columns)
#df2["LUC"] =  np.where(((df2['LUC']==9) & (df2['SUB']==0)),8,df['LUC'])
# sum land area by subcatchment and LUC
def fn_sum_by_sub_and_luc(df_SUB_LUC_Area1,df_SUB_LUC,df_SUB_LUC_Area2):
    print(df_SUB_LUC)
    df2_SUB_LUC_merged = pd.merge(df_SUB_LUC,df_SUB_LUC_Area1,'left')
    print(df_SUB_LUC)
    Area_Ha = df2_SUB_LUC_merged['Area_Ha'].fillna(0) + df_SUB_LUC_Area2['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    return(df_SUB_LUC_AREA)

df2_SUB_CC_Area = df2.groupby(['SUB','LUC'])['Area_Ha'].sum().reset_index(name='Area_Ha')
print(df2_SUB_CC_Area)
print(df2_SUB_CC_Area.groupby('LUC')['Area_Ha'].sum())
print(df_SUB_LUC)
print(df_SUB_LUC_merged)
df2_i = fn_sum_by_sub_and_luc(df2_SUB_CC_Area,df_SUB_LUC,df_SUB_LUC_merged)
df2_i.to_csv('df2_SUB_CC_Area.csv')
print(df_SUB_LUC)

Number of LUC == 13: 20382
Index(['Unnamed: 0', 'FID', 'FID_out_gg', 'FID_LANDCO', 'COVERNAME',
       'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE', 'POLY_TYPE', 'FY',
       'TOWN_ID', 'TILENAME', 'Shape_Leng', 'Shape_Area', 'FID_Great_', 'SUB',
       'SHED_NAME', 'Shape_ar_1', 'Shape_len', 'Terminus', 'Area_Ha',
       'FID_in_ggb', 'ID', 'WMA_NO', 'OWNER', 'ADDRESS', 'TOWN', 'REGION',
       'BOG_NAME', 'REGAREA', 'CERTAREA', 'CREDITAREA', 'PERMITAREA',
       'TOTAREA', 'STAFF', 'PROGRAM', 'DATE_ENTER', 'COMMENT', 'BIRTHREG',
       'AREA', 'PERIMETER', 'PERMIT_NUM', 'OWNER_FIRS', 'OWNER_LAST',
       'AREASACRES', 'Basin', 'CropStatus', 'BOG_FID', 'Shape_Le_1',
       'Shape_Ar_2', 'Bog_type', 'ResidentialUnit', 'LUC'],
      dtype='object')
     SUB  LUC     Area_Ha
0      1    1  379.205071
1      1    2  154.025886
2      1    6    0.343068
3      1    7   44.660720
4      1   10    0.139275
..   ...  ...         ...
224   33    6    3.521194
225   33    7    5.271894
2

In [344]:
df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
df_SUB_LUC_merged_general = df_SUB_LUC_merged
df_SUB_LUC_merged_general['Area_Ha'] = 0
print(df_SUB_LUC_merged_general)
df2_i = fn_sum_by_sub_and_luc(df2_SUB_CC_Area,df_SUB_LUC,df_SUB_LUC_merged_general).dropna()
print(df2_i)
# Aggregate Data with Pandas .groupby()
df2_i.to_csv(os.path.join(odr,"Wareham_LUC_Area_general"+".csv"))
display(df2_i)
# check that area is within one tenth of a Ha of input
print('Difference in Total Area (Ha):',round(df2['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

     SUB  LUC  Area_Ha
0      4    1        0
1      6    2        0
2      2    3        0
3      5    4        0
4      3    5        0
..   ...  ...      ...
430   31    9        0
431   24   10        0
432   27   11        0
433   32   12        0
434   30   13        0

[435 rows x 3 columns]
     SUB  LUC
0      4    1
1      6    2
2      2    3
3      5    4
4      3    5
..   ...  ...
424   31    9
425   24   10
426   27   11
427   32   12
428   30   13

[429 rows x 2 columns]
     SUB  LUC
0      4    1
1      6    2
2      2    3
3      5    4
4      3    5
..   ...  ...
424   31    9
425   24   10
426   27   11
427   32   12
428   30   13

[429 rows x 2 columns]
4582.646524428546
435
     SUB  LUC    Area_Ha
0      4    1  47.368926
1      6    2   1.838545
2      2    3   0.000000
3      5    4   0.000000
4      3    5   0.000000
..   ...  ...        ...
424   31    9   0.000000
425   24   10  18.817178
426   27   11   0.000000
427   32   12   0.000000
428   30   13   0.4

,SUB,LUC,Area_Ha
0,4,1,47.368926
1,6,2,1.838545
2,2,3,0.000000
3,5,4,0.000000
4,3,5,0.000000
...,...,...,...
424,31,9,0.000000
425,24,10,18.817178
426,27,11,0.000000
427,32,12,0.000000


Difference in Total Area (Ha): 0.0


Reclassify terminus and land use based on cranberry management scenarios

In [345]:
display(df2)
display(df1)
# List of subwatersheds with cranberry at terminus
SUB_LIST_1 = [19,26,29]
# List of subwatersheds with flowthrough cranberry at terminus
SUB_LIST_4 = [19,26,29]
# List of cranberry bogs at terminus of subwatersheds
#BOG_FID_LIST_1 = [4,7,11]
BOG_FID_LIST_1 = [167,584,977]
BOG_FID_LIST_1 = [str(i) for i in BOG_FID_LIST_1]
print("total restored area (Ha):",df2[np.isin(df2['BOG_FID'],BOG_FID_LIST_1)]['Area_Ha'].sum())
print(BOG_FID_LIST_1)

,Unnamed: 0,FID,FID_out_gg,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_Great_,SUB,SHED_NAME,Shape_ar_1,Shape_len,Terminus,Area_Ha,FID_in_ggb,ID,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Shape_Le_1,Shape_Ar_2,Bog_type,ResidentialUnit,LUC
0,0,0,0,19202091,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,59.355339,178.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017850,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1
1,1,1,1,19202092,Developed Open Space,5,Tax exempt,9,900,FEE,2011,261,R13C25,497.697006,1115.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.111562,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,2
2,2,2,2,19202150,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,182.752309,593.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.059350,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1
3,3,3,3,19202201,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,226.793939,1051.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.105150,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1
4,4,4,4,19202221,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,62.577164,178.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017862,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84220,84220,84220,82317,22780276,Cultivated,6,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,499.295570,7705.037920,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.762434,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,NaN,42209609,238739.793,8156.279,NaN,NaN,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough,False,4
84221,84221,84221,82666,22781347,Palustrine Emergent Wetland,15,Open land,2,932,FEE,2017,96,R14C25,4.276612,0.530611,25,19,Lower Coonamessett River,3.521906e+06,11291.923980,s,0.000053,6,0,42209610,Town of Falmouth,Old Meeting House Rd.,Falmouth,4,Middle/lower bog,24.4,0.0,0.0,0.0,24.43,Jim McLaughlin,BRP_WMA,2009-05-13,NaN,42209609,98550.168,4547.228,NaN,NaN,Town of Falmouth,24.352,Cape Cod,active,584,4546.272832,98508.748324,flowthrough,False,4
84222,84222,84222,82836,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,20,16,Flax Pond LT10,7.571123e+05,5773.200642,p,0.029896,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,NaN,42209609,238739.793,8156.279,NaN,NaN,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough,False,4
84223,84223,84223,82837,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.018259,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,NaN,42209609,238739.793,8156.279,NaN,NaN,Handy Cranberry Trust,58.994,Cape Cod,active,167,

,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,4,Deep Pond GT10,p,71.783290,0.000000,0.012525,0,0.000000
1,6,Crooked Pond GT10,p,53.071712,0.001371,0.003341,21,0.001371
2,2,Coonamessett Pond LT10,p,352.846203,0.020308,0.033739,311,0.020308
3,5,Deep Pond LT10,p,65.209861,0.007183,0.007008,110,0.007183
4,3,Round Pond,p,37.019734,0.001306,0.002674,20,0.001306
5,7,Crooked Pond LT10,p,78.128171,0.022463,0.016022,344,0.022463
6,9,Round Pond,p,87.236109,0.007575,0.010214,116,0.007575
7,15,Flax Pond GT10,p,22.884859,0.001437,0.004489,22,0.001437
8,12,Mares Pond GT10,p,54.522097,0.002808,0.004587,43,0.002808
9,10,Jenkins Pond,p,213.377284,0.039441,0.028509,604,0.039441


total restored area (Ha): 42.487701015356734
['167', '584', '977']


In [346]:
# Dictionary of Scenario Conditions
LUC_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: 'False',
    #1: "Restore Select Bogs to Wetlands"
    1: "np.isin(df2['BOG_FID'],BOG_FID_LIST_1)",
    #2: "Restore All Retired Bogs to Wetlands"
    2: "np.isin(df2['FID'],df2[df2.CropStatus == 'abandoned']['FID'])",
    #3: "Retire All Flowthrough Bogs"
    3: "np.isin(df2['FID'],df2[df2.Bog_type == 'flowthrough']['FID'])",
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: "np.isin(df2['FID'],df2[df2.Bog_type == 'flowthrough']['FID'])"
}
Terminus_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: '[False]',
    #1: "Restore Select Bogs to Wetlands"
    1: 'np.isin(df1[\'SUB\'],SUB_LIST_1)',
    #2: "Restore All Retired Bogs to Wetlands"
    2: '[False]',
    #3: "Retire All Flowthrough Bogs"
    3: '[False]',
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: 'np.isin(df1[\'SUB\'],SUB_LIST_4)'
}
#df2 = df2.replace(r'^\s*$', np.nan, regex=True)
#display(df2)
#True in [i is 'abandoned' for i in df2.CropStatus]
Areasum = df2[df2.CropStatus == 'abandoned'].groupby(['SHED_NAME','LUC'])['Area_Ha'].sum()
print(Areasum)
print(df2[df2.FID==59495]['CropStatus'] == 'abandoned')
print(df2[df2.FID==114647]['Bog_type'] == 'flowthrough')
#print([i is "abandoned" for i in df2.CropStatus.astype("|S")])
print(df2[['BOG_FID','CropStatus','SUB']].dtypes)
print(eval(LUC_scenarios[2]))
df2.BOG_FID.to_csv("BOG_FID.csv")
print("Is 'True' in the result: ",True in eval(Terminus_scenarios[4]))
Terminus_scenarios.keys()
Terminus_scenarios.values()

Series([], Name: Area_Ha, dtype: float64)
59495    False
Name: CropStatus, dtype: bool
Series([], Name: Bog_type, dtype: bool)
BOG_FID       object
CropStatus    object
SUB            int64
dtype: object
[False False False ... False False False]
Is 'True' in the result:  True


dict_values(['[False]', "np.isin(df1['SUB'],SUB_LIST_1)", '[False]', '[False]', "np.isin(df1['SUB'],SUB_LIST_4)"])

In [347]:
# For each scenario reclassify land use, then sum up area by land use and subwatershed
# default value if scenario condition is true
LUC_iftrue = 10
Terminus_iftrue = "w"
scens = [str(i) for i in range(5)]
list(LUC_scenarios.keys())

[0, 1, 2, 3, 4]

In [348]:
# For each scenario reclassify land use,
out_df2 = df2
out_df1 = df1
# switch Terminus of Maple Swamp from w to s
#rownums = np.where(out_df1['SubW_Name']=='Maple Swamp')[0]
#out_df1.at[rownums,'Terminus']='s'
# default LUC value if scenario condition is true
LUC_iftrue = 10
# default Terminus value
Terminus_iftrue = "w"
for i in list(LUC_scenarios.keys()):
    print (i)
    # alternative land use for scenario 
    if i == 3: 
        LUC_iftrue = 11
    else: 
        LUC_iftrue = 10
        
    # alter LUC based on scenario conditions
    #np.where(logic,if_true, if_false)
    out_df2["LUC"+str(i)] = np.where(eval(LUC_scenarios[i]),LUC_iftrue,df2.LUC)
    sum((np.where(eval(LUC_scenarios[i]),LUC_iftrue,df2.LUC)))
    out_df1["Terminus"+str(i)] = np.where(eval(Terminus_scenarios[i]),Terminus_iftrue,df1.Terminus)
display(out_df2)
display(out_df1)

print(out_df2.groupby('LUC0')['Area_Ha'].sum())
print(out_df2.groupby('LUC1')['Area_Ha'].sum())
print(out_df2.groupby('LUC2')['Area_Ha'].sum())
print(out_df2.groupby('LUC3')['Area_Ha'].sum())
print(out_df2.groupby('LUC4')['Area_Ha'].sum())

#Save the scenario reclassifications to csv
out_path1 = os.path.join(odr,inname1+'_reclass.csv')
out_path2 = os.path.join(odr,inname2+'_reclass.csv')
out_df1.to_csv(out_path1)
out_df2.to_csv(out_path2)

0
1
2
3
4


,Unnamed: 0,FID,FID_out_gg,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_Great_,SUB,SHED_NAME,Shape_ar_1,Shape_len,Terminus,Area_Ha,FID_in_ggb,ID,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Shape_Le_1,Shape_Ar_2,Bog_type,ResidentialUnit,LUC,LUC0,LUC1,LUC2,LUC3,LUC4
0,0,0,0,19202091,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,59.355339,178.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017850,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1,1,1,1,1,1
1,1,1,1,19202092,Developed Open Space,5,Tax exempt,9,900,FEE,2011,261,R13C25,497.697006,1115.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.111562,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,2,2,2,2,2,2
2,2,2,2,19202150,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,182.752309,593.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.059350,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1,1,1,1,1,1
3,3,3,3,19202201,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,226.793939,1051.500000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.105150,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1,1,1,1,1,1
4,4,4,4,19202221,Deciduous Forest,9,Tax exempt,9,900,FEE,2011,261,R13C25,62.577164,178.625000,16,1,Coonamessett Pond GT10,6.239457e+06,20696.124556,p,0.017862,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.00,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000000,0.000000,NaN,False,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84220,84220,84220,82317,22780276,Cultivated,6,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,499.295570,7705.037920,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.762434,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,NaN,42209609,238739.793,8156.279,NaN,NaN,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough,False,4,4,10,4,11,10
84221,84221,84221,82666,22781347,Palustrine Emergent Wetland,15,Open land,2,932,FEE,2017,96,R14C25,4.276612,0.530611,25,19,Lower Coonamessett River,3.521906e+06,11291.923980,s,0.000053,6,0,42209610,Town of Falmouth,Old Meeting House Rd.,Falmouth,4,Middle/lower bog,24.4,0.0,0.0,0.0,24.43,Jim McLaughlin,BRP_WMA,2009-05-13,NaN,42209609,98550.168,4547.228,NaN,NaN,Town of Falmouth,24.352,Cape Cod,active,584,4546.272832,98508.748324,flowthrough,False,4,4,10,4,11,10
84222,84222,84222,82836,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,20,16,Flax Pond LT10,7.571123e+05,5773.200642,p,0.029896,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,0.0,59.20,Jim McLaughlin,BRP_WMA,2007-09-13,NaN,42209609,238739.793,8156.279,NaN,NaN,Handy Cranberry Trust,58.994,Cape Cod,active,167,8075.184456,238638.517506,flowthrough,False,4,4,10,4,11,10
84223,84223,84223,82837,22781979,Grassland,8,"Mixed use, primarily residential",10,017,FEE,2017,96,R14C25,384.854927,1653.698600,24,26,Backus Brook LT10,1.754130e+06,8147.386465,s,0.018259,3,0,42209609,Handy Cranberry Trust,Route 28,Falmouth,4,Bachus Bog,59.0,0.0,0.0,

,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,Terminus0,Terminus1,Terminus2,Terminus3,Terminus4
0,4,Deep Pond GT10,p,71.783290,0.000000,0.012525,0,0.000000,p,p,p,p,p
1,6,Crooked Pond GT10,p,53.071712,0.001371,0.003341,21,0.001371,p,p,p,p,p
2,2,Coonamessett Pond LT10,p,352.846203,0.020308,0.033739,311,0.020308,p,p,p,p,p
3,5,Deep Pond LT10,p,65.209861,0.007183,0.007008,110,0.007183,p,p,p,p,p
4,3,Round Pond,p,37.019734,0.001306,0.002674,20,0.001306,p,p,p,p,p
5,7,Crooked Pond LT10,p,78.128171,0.022463,0.016022,344,0.022463,p,p,p,p,p
6,9,Round Pond,p,87.236109,0.007575,0.010214,116,0.007575,p,p,p,p,p
7,15,Flax Pond GT10,p,22.884859,0.001437,0.004489,22,0.001437,p,p,p,p,p
8,12,Mares Pond GT10,p,54.522097,0.002808,0.004587,43,0.002808,p,p,p,p,p
9,10,Jenkins Pond,p,213.377284,0.039441,0.028509,604,0.039441,p,p,p,p,p


LUC0
1     2524.257418
2      712.406116
4       49.978002
5        8.452357
6      164.589834
7      321.358501
9      428.785737
10     185.581154
13     187.237405
Name: Area_Ha, dtype: float64
LUC1
1     2524.257418
2      712.406116
4        7.490301
5        8.452357
6      164.589834
7      321.358501
9      428.785737
10     228.068855
13     187.237405
Name: Area_Ha, dtype: float64
LUC2
1     2524.257418
2      712.406116
4       49.978002
5        8.452357
6      164.589834
7      321.358501
9      428.785737
10     185.581154
13     187.237405
Name: Area_Ha, dtype: float64
LUC3
1     2524.257418
2      712.406116
5        8.452357
6      164.589834
7      321.358501
9      428.785737
10     185.581154
11      49.978002
13     187.237405
Name: Area_Ha, dtype: float64
LUC4
1     2524.257418
2      712.406116
5        8.452357
6      164.589834
7      321.358501
9      428.785737
10     235.559155
13     187.237405
Name: Area_Ha, dtype: float64


In [349]:
#copy the original layer and add to the map
#fc1_reclass = arcpy.management.CopyFeatures(fc1, fc1+'_reclass')
fc2_reclass = arcpy.management.CopyFeatures(inname2, inname2+'_reclass')
print(fc2_reclass)
# Join the scenarios csv files with the copy of original layers
#arcpy.management.AddJoin(fc1_reclass, "FID", out_path1, "FID", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
fc2_reclass = arcpy.management.AddJoin(fc2_reclass, "FID", out_path2, "FID", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")



C:\Workspace\Geodata\Nload\out_lclu16_ggbp_cran_ident_reclass.shp


In [350]:
# Conditions for terminus transmission coefficient
df1_dict = {'w': 0.23,
'c': 1,
'e': 1,
's': 1,
'p': 1}
df1_conditions = '['+', '.join(['df1_i.Terminus =="' + k+'"' for k in df1_dict.keys()])+']'
df2_dict = {'9': 1,
'10': 1}
df2_conditions = '['+', '.join(['df2_i.Terminus =="' + k+'"' for k in df2_dict.keys()])+']'
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"GGBP_Terminus_and_Pop"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    """    
    df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
    display(df_SUB_LUC)
    display(df_SUB_LUC_merged)
    df2_i = pd.merge(df_SUB_LUC,df2_i,'left')
    Area_Ha = df_SUB_LUC_merged['Area_Ha'].fillna(0) + df2_i['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    df2_i = pd.merge(df_SUB_LUC_AREA,df1_i[['SUB','Terminus']])
    df2_i['c_T'] = np.select(eval(df2_conditions),df2_dict.values(),default=1)
    print("!!!!!!!!!!!!displaying df2_i after merge")
    """
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"GGBP_LUC_Area_p1_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,1.0
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,1.0
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,1.0
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,1.0
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,1.0
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,1.0
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,1.0
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,1.0
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,1.0
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,1.0


,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
224,33,6,3.521194
225,33,7,5.271894
226,33,9,4.390090
227,33,10,3.658271


Difference in Total Area (Ha): 0.0
1


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,1.00
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,1.00
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,1.00
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,1.00
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,1.00
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,1.00
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,1.00
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,1.00
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,1.00
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,1.00


...

,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
224,33,6,3.521194
225,33,7,5.271894
226,33,9,4.390090
227,33,10,3.658271


Difference in Total Area (Ha): 0.0
4


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,1.00
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,1.00
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,1.00
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,1.00
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,1.00
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,1.00
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,1.00
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,1.00
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,1.00
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,1.00


,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
219,33,6,3.521194
220,33,7,5.271894
221,33,9,4.390090
222,33,10,3.658271


Difference in Total Area (Ha): 0.0


In [351]:
df1_dict = {'w': 0.23,
'c': 1,
'e': 1,
's': 1,
'p': 0.5}
df1_conditions = '['+', '.join(['df1_i.Terminus =="' + k+'"' for k in df1_dict.keys()])+']'
df2_dict = {'9': 1,
'10': 1}
df2_conditions = '['+', '.join(['df2_i.Terminus =="' + k+'"' for k in df2_dict.keys()])+']'
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"GGBP_Terminus_and_Pop"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    '''
    df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
    display(df_SUB_LUC)
    display(df_SUB_LUC_merged)
    df2_i = pd.merge(df_SUB_LUC,df2_i,'left')
    Area_Ha = df_SUB_LUC_merged['Area_Ha'].fillna(0) + df2_i['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    df2_i = pd.merge(df_SUB_LUC_AREA,df1_i[['SUB','Terminus']])
    df2_i['c_T'] = np.select(eval(df2_conditions),df2_dict.values(),default=1)
    print("!!!!!!!!!!!!displaying df2_i after merge")
    '''
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"GGBP_LUC_Area_p05_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,0.5
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,0.5
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,0.5
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,0.5
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,0.5
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,0.5
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,0.5
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,0.5
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,0.5
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,0.5


,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
224,33,6,3.521194
225,33,7,5.271894
226,33,9,4.390090
227,33,10,3.658271


Difference in Total Area (Ha): 0.0
1


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,0.50
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,0.50
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,0.50
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,0.50
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,0.50
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,0.50
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,0.50
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,0.50
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,0.50
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,0.50


...

,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
224,33,6,3.521194
225,33,7,5.271894
226,33,9,4.390090
227,33,10,3.658271


Difference in Total Area (Ha): 0.0
4


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000,0.50
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371,0.50
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308,0.50
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183,0.50
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306,0.50
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463,0.50
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575,0.50
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437,0.50
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808,0.50
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441,0.50


,SUB,LUC,Area_Ha
0,1,1,379.205071
1,1,2,154.025886
2,1,6,0.343068
3,1,7,44.660720
4,1,10,0.139275
...,...,...,...
219,33,6,3.521194
220,33,7,5.271894
221,33,9,4.390090
222,33,10,3.658271


Difference in Total Area (Ha): 0.0


In [352]:
# GENERAL 
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    #df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"GGBP_Terminus_and_Pop_"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    df2_i = pd.merge(df2_i,df1_i[['SUB','Terminus']],'inner')
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"GGBP_LUC_Area_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,4,p,Deep Pond GT10,71.783290,0.000000,0.012525,0,0.000000
1,6,p,Crooked Pond GT10,53.071712,0.001371,0.003341,21,0.001371
2,2,p,Coonamessett Pond LT10,352.846203,0.020308,0.033739,311,0.020308
3,5,p,Deep Pond LT10,65.209861,0.007183,0.007008,110,0.007183
4,3,p,Round Pond,37.019734,0.001306,0.002674,20,0.001306
5,7,p,Crooked Pond LT10,78.128171,0.022463,0.016022,344,0.022463
6,9,p,Round Pond,87.236109,0.007575,0.010214,116,0.007575
7,15,p,Flax Pond GT10,22.884859,0.001437,0.004489,22,0.001437
8,12,p,Mares Pond GT10,54.522097,0.002808,0.004587,43,0.002808
9,10,p,Jenkins Pond,213.377284,0.039441,0.028509,604,0.039441


PermissionError: [Errno 13] Permission denied: 'C:\\Workspace\\Geodata\\Nload\\outputs\\GGBP_Terminus_and_Pop_0.csv'





















## Appendix

Generate a table of area for 13 land use classes within each subcatchment, by sorting land uses from the following categories. 

Class Number|	Class Name
---|---
2|	Impervious
5|	Developed Open Space
6|	Cultivated Land
7|	Pasture/Hay
8|	Grassland
9|	Deciduous Forest
10|	Evergreen Forest
12|	Scrub/Shrub
13|	Palustrine Forested Wetland (C-CAP)
14|	Palustrine Scrub/Shrub Wetland (C-CAP)
15|	Palustrine Emergent Wetland (C-CAP)
16|	Estuarine Forested Wetland (C-CAP)
17|	Estuarine Scrub/Shrub Wetland (C-CAP)
18|	Estuarine Emergent Wetland (C-CAP)
19|	Unconsolidated Shore
20|	Bare Land
21|	Open Water
22|	Palustrine Aquatic Bed (C-CAP)
23|	Estuarine Aquatic Bed (C-CAP)


USEGENCODE|	USEGENNAME
---|---
0|	Unknown
2|	Open land
3|	Commercial
4|	Industrial
6|	Forest
7|	Agriculture
8|	Recreation
9|	Tax exempt
10| Mixed use, primarily residential
11|	Residential - single family
12|	Residential - multi-family
13|	Residential - other
20|	Mixed use, other
30|	Mixed use, primarily commercial
55|	Right-of-way
88|	Water